In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 1) sanity: GPU present?
!nvidia-smi

# 2) fresh pip + basics
!pip -q install --upgrade pip

# 3) (Optional pin) Use the preinstalled torch on Colab (often 2.x + cu121). Print it:
import torch, os, sys
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda, "Py", sys.version)

# 4) Install PyG wheels that match THIS torch build (auto-picks correct index page)
os.environ['TORCH_VER'] = torch.__version__
!pip -q install "torch-scatter" "torch-sparse" "torch-cluster" "torch-spline-conv" "torch-geometric" -f https://data.pyg.org/whl/torch-$TORCH_VER.html

# 5) Other deps
!pip -q install transformers tqdm


Wed Sep  3 02:41:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
import os
os.makedirs('/content/Thesis-causal-vul', exist_ok=True)

In [12]:
%cd /content/Thesis-causal-vul

/content/Thesis-causal-vul


In [19]:
!python -m src.step4_train_gnn_attn \
  --data_dir "/content/Thesis-causal-vul/work/pyg_diversevul_tf" \
  --out_dir "/content/Thesis-causal-vul/work/models" \
  --epochs 80 \
  --batch_size 64 \
  --hidden 192 \
  --attn_hidden 192 \
  --lr 5e-4 \
  --loss focal \
  --gamma 2.0 \
  --sampler weighted \
  --progress tqdm \
  --num_workers 2 \
  --clean_out



[labels] train pos=10 neg=1018
[loss] Focal(gamma=2.0, alpha_pos=0.990)
[01] train_loss=3.9963  val_loss=0.0870  val_acc=0.000  lr=1.40e-04
[02] train_loss=1.2844  val_loss=0.1183  val_acc=0.230  lr=2.30e-04
[03] train_loss=0.3074  val_loss=0.1582  val_acc=0.475  lr=3.20e-04
[04] train_loss=0.1064  val_loss=0.1121  val_acc=0.705  lr=4.10e-04
/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
[05] train_loss=0.0560  val_loss=0.0731  val_acc=0.871  lr=5.00e-04
[06] train_loss=0.0339  val_loss=0.0959

In [22]:
!python -m src.step5_export_chains \
  --data_dir "/content/Thesis-causal-vul/work/pyg_diversevul_tf" \
  --model   "/content/Thesis-causal-vul/work/models/gcbert_gnn_attn.pt" \
  --cfg     "/content/Thesis-causal-vul/work/models/gcbert_gnn_attn.cfg.json" \
  --out     "/content/Thesis-causal-vul/work/chains/colab_chains.jsonl" \
  --num_graphs 500 \
  --topk_nodes 12 \
  --k_paths 3 \
  --max_len 8 \
  --clean_out


[ok] wrote /content/Thesis-causal-vul/work/chains/colab_chains.jsonl


In [23]:
!pip -q install scikit-learn
!python -m src.step6_eval_report \
  --data_dir "/content/Thesis-causal-vul/work/pyg_diversevul_tf" \
  --model   "/content/Thesis-causal-vul/work/models/gcbert_gnn_attn.pt" \
  --cfg     "/content/Thesis-causal-vul/work/models/gcbert_gnn_attn.cfg.json" \
  --out_dir "/content/Thesis-causal-vul/work/eval" \
  --seed 7


{
  "n_test": 115,
  "default_thr": 0.5,
  "metrics@0.5": {
    "tp": 1,
    "tn": 110,
    "fp": 4,
    "fn": 0,
    "acc": 0.9652173913043478,
    "prec": 0.2,
    "rec": 1.0,
    "f1": 0.33333333333333337,
    "roc_auc": 1.0,
    "pr_auc": 1.0
  },
  "best_threshold_by_f1": {
    "thr": 0.9,
    "f1": 1.0
  },
  "metrics@best_thr": {
    "tp": 1,
    "tn": 114,
    "fp": 0,
    "fn": 0,
    "acc": 1.0,
    "prec": 1.0,
    "rec": 1.0,
    "f1": 1.0,
    "roc_auc": 1.0,
    "pr_auc": 1.0
  }
}


In [24]:
!python -m src.step6_render_chains_html \
  --chains "/content/Thesis-causal-vul/work/chains/colab_chains.jsonl" \
  --out_html "/content/Thesis-causal-vul/work/chains/colab_chains.html" \
  --min_prob 0.5 \
  --max_rows 300

# Preview (openable link)
from google.colab import files, output
print("Open:", "file:///content/Thesis-causal-vul/work/chains/colab_chains.html")


[ok] wrote /content/Thesis-causal-vul/work/chains/colab_chains.html (rows=15)
Open: file:///content/Thesis-causal-vul/work/chains/colab_chains.html
